# Ergodicity Basic Simulation - Error Verification

In [1]:
error_pickle_list = [
    'Cap (10M) -    ATR (0.8) -    EBITABL (0.125) -    XS_Kurt (952839) -    NOINS -    100K Sims -    50 Yrs.pkl',
    'Cap (10M) -    ATR (1.2) -    EBITABL (0.1) -    XS_Kurt (6209) -    Ded (500K) -    LR (0.8) -    100K Sims -    50 Yrs.pkl',
    'Cap (10M) -    ATR (1.2) -    EBITABL (0.125) -    XS_Kurt (7611) -    NOINS -    100K Sims -    50 Yrs.pkl',
    'Cap (25M) -    ATR (0.9) -    EBITABL (0.15) -    XS_Kurt (2397) -    Ded (0K) -    LR (0.7) -    100K Sims -    50 Yrs.pkl',
    'Cap (10M) -    ATR (0.8) -    EBITABL (0.1) -    XS_Kurt (10938) -    Ded (50K) -    LR (0.6) -    100K Sims -    50 Yrs.pkl',
    'Cap (25M) -    ATR (1.0) -    EBITABL (0.1) -    XS_Kurt (10353) -    Ded (500K) -    LR (0.8) -    100K Sims -    50 Yrs.pkl',
    'Cap (25M) -    ATR (1.0) -    EBITABL (0.15) -    XS_Kurt (10505) -    NOINS -    100K Sims -    50 Yrs.pkl',
    'Cap (25M) -    ATR (1.0) -    EBITABL (0.15) -    XS_Kurt (3816128) -    Ded (250K) -    LR (0.7) -    100K Sims -    50 Yrs.pkl',
    'Cap (25M) -    ATR (1.0) -    EBITABL (0.15) -    XS_Kurt (6365) -    NOINS -    100K Sims -    50 Yrs.pkl',
    'Cap (25M) -    ATR (1.2) -    EBITABL (0.125) -    XS_Kurt (14616) -    Ded (0K) -    LR (0.8) -    100K Sims -    50 Yrs.pkl',
    'Cap (5M) -    ATR (0.9) -    EBITABL (0.1) -    XS_Kurt (2114) -    NOINS -    100K Sims -    50 Yrs.pkl',
    'Cap (5M) -    ATR (0.9) -    EBITABL (0.1) -    XS_Kurt (2361) -    Ded (50K) -    LR (0.6) -    100K Sims -    50 Yrs.pkl',
    'Cap (5M) -    ATR (1.1) -    EBITABL (0.125) -    XS_Kurt (1689) -    NOINS -    100K Sims -    50 Yrs.pkl',
    'Cap (5M) -    ATR (1.1) -    EBITABL (0.125) -    XS_Kurt (46400) -    Ded (250K) -    LR (0.6) -    100K Sims -    50 Yrs.pkl',
    'Cap (5M) -    ATR (1.1) -    EBITABL (0.15) -    XS_Kurt (2921) -    Ded (0K) -    LR (0.6) -    100K Sims -    50 Yrs.pkl',
    'Cap (5M) -    ATR (1.2) -    EBITABL (0.15) -    XS_Kurt (55801) -    Ded (50K) -    LR (0.7) -    100K Sims -    50 Yrs.pkl'
]

error_pickle_list
import pickle
import re

from pathlib import Path
from PIL import Image

from ergodic_insurance.monte_carlo import SimulationResults

def _parse_number(text):
    text = text.strip().replace(",", "")
    m = re.fullmatch(r'([+-]?\d+(?:\.\d+)?)([KMB])?$', text, re.I)
    if not m:
        # fallback: plain int/float or leave as-is
        try:
            return int(text)
        except ValueError:
            try:
                return float(text)
            except ValueError:
                return text
    num = float(m.group(1))
    mult = {"K": 1_000, "M": 1_000_000, "B": 1_000_000_000}.get((m.group(2) or "").upper(), 1)
    val = num * mult
    return int(val) if val.is_integer() else val


def parse_config_key(key: str) -> dict:
    parts = re.split(r"\s*-\s*", key.strip())
    out = {}
    for part in parts:
        if not part:
            continue

        # e.g. "Cap (100M)"
        m = re.match(r"^([A-Za-z_]+)\s*\(\s*([^)]+)\s*\)$", part)
        if m:
            out[m.group(1)] = _parse_number(m.group(2))
            continue

        # e.g. "0K Sims" or "50 Yrs"
        m = re.match(r"^([+-]?\d+(?:\.\d+)?)\s*([KMB])?\s*([A-Za-z_]+)$", part)
        if m:
            value = _parse_number((m.group(1) or "") + (m.group(2) or ""))
            out[m.group(3)] = value
            continue

        # flags like "NOINS"
        if part.upper() == "NOINS":
            out["NOINS"] = True

    return out

results_dir = Path("G:/My Drive/Colab Notebooks/results")

# Build concrete paths from the known filenames and report any missing files
pkl_paths = []
missing_files = []
for fname in error_pickle_list:
    p = results_dir / fname
    if p.exists():
        pkl_paths.append(p)
    else:
        missing_files.append(fname)
if missing_files:
    print(f"Missing {len(missing_files)} files under {results_dir}:")
    for mf in missing_files:
        print(f" - {mf}")

error_configurations = {}
if not pkl_paths:
    print(f"No pickle files found in {results_dir}.")
else:
    for path in pkl_paths:
        try:
            with open(path, "rb") as f:
                one_config = pickle.load(f)
                growth_rate = one_config.growth_rates.mean()
                ror = one_config.ruin_probability
                error_configurations[path.stem] = {
                    "growth_rate": growth_rate,
                    "risk_of_ruin": ror
                }
        except Exception as e:
            print(f"Skipping {path.name}: {e}")
    print(f"Loaded {len(error_configurations)} pickle files into all_configurations.")

parsed_params_by_key_errors = {k: parse_config_key(k) | error_configurations[k] for k in error_configurations.keys()}
parsed_params_by_key_errors

KeyboardInterrupt: 